# Time for a Test Drive!

You've spent some time walking around the Dascar lot, hearing about all the awesome features and specs...

That's enough talk! Let's jump into a racecar and see what it can do!

<img src="files/racecar.gif">

## Dask DataFrames

The pandas car...with the Dask engine!

In [ ]:
import dask.dataframe as dd

In [ ]:
%run ../prep_data.py -d flights

In [ ]:
import os

files = os.path.join('../data', 'nycflights', '*.csv')
files

In [ ]:
df = dd.read_csv(files,
                 parse_dates={'Date': [0, 1, 2]},
                 dtype={"TailNum": str,
                        "CRSElapsedTime": float,
                        "Cancelled": bool})

In [ ]:
df.head()

In [ ]:
%%time
df.groupby("Origin")["DepDelay"].mean().compute()

In [ ]:
df

## Dask Arrays

The Numpy car...with Dask engine superpowers!

In [ ]:
import dask.array as da

In [ ]:
array = da.random.random((10000, 10000), chunks=(1000, 1000))

In [ ]:
array

In [ ]:
array[:10,:5]

In [ ]:
array[:10,:5].compute()

In [ ]:
%%time
array.sum(axis=1).compute()

## Dask ML

The scikit-learn car with.... you guessed it -- Dask rocketfuel!

In [ ]:
from dask_ml.linear_model import LogisticRegression
from dask_ml.datasets import make_classification

In [ ]:
X, y = make_classification(n_samples=1_000, chunks=50)

In [ ]:
X

In [ ]:
y

In [ ]:
lr = LogisticRegression()

In [ ]:
%%time
lr.fit(X, y)

In [ ]:
%%time
predictions = lr.predict(X).compute()

In [ ]:
lr.score(X,y).compute()

## Custom Dask

In [ ]:
from time import sleep

def inc(x):
    """Increments x by one"""
    sleep(1)
    return x + 1

def add(x=0, y=0, z=0):
    """Adds x and y and z"""
    sleep(1)
    return x + y + z

In [ ]:
%%time

x = inc(1)
y = inc(2)
z = add(x, y)

In [ ]:
z

In [ ]:
from dask import delayed

In [ ]:
%%time

a = delayed(inc)(1)
b = delayed(inc)(2)
c = delayed(add)(a, b)

In [ ]:
c

In [ ]:
a.visualize()

In [ ]:
b.visualize()

In [ ]:
c.visualize()

In [ ]:
d = delayed(inc)(3)

In [ ]:
c = delayed(add)(a, b, d)

In [ ]:
c.visualize()

In [ ]:
%%time
c.compute()

## Dask Cluster on Coiled

In [ ]:
import coiled

In [ ]:
cluster = coiled.Cluster(
    name="dask-mini-tutorial", 
    n_workers=20, 
    worker_memory='25Gib',
    software="rrpelgrim/dask-mini-tutorial",
    scheduler_options={'idle_timeout':'3 hours'},
    shutdown_on_close=False,
)

In [ ]:
from distributed import Client

client = Client(cluster)
client

In [ ]:
import dask.dataframe as dd

In [ ]:
df = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        "payment_type": "UInt8",
        "VendorID": "UInt8",
        "passenger_count": "UInt8",
        "RatecodeID": "UInt8",
        "store_and_fwd_flag": "category",
        "PULocationID": "UInt16",
        "DOLocationID": "UInt16",
    },
    storage_options={"anon": True},
    blocksize="16 MiB",
)

In [ ]:
df

In [ ]:
df.groupby("passenger_count").tip_amount.mean().compute()